In [1]:
from Annotation_helper import *

Sets correct values for generated figures:

In [2]:
#Set all settings for the resulting figures

import seaborn as sns
sns.set()

SMALL_SIZE = 15
MEDIUM_SIZE = 17
BIGGER_SIZE = 19

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.tight_layout()

Set paths to files

In [3]:
data_folder = "../../Data"
structured_data_folder = data_folder + "/structured_data"
text_path = data_folder + "/text"
data_path = structured_data_folder + "/test.csv"
dist_folder = structured_data_folder + "/Distances"
figure_folder = "../Figures/Results/Embeddings"
model_folder = "../Models/Embeddings"
matches = pd.read_csv(structured_data_folder + "/Matches/Bas.csv", index_col=0)
figure_folder = "../Figures/Results/Classic"

In [4]:
transactions = load_transactions("../../Data/structured_data\VOC Cochin Slave Transactions 1706-1801 - IISH - Database - 2018 - v1.csv")
permissions = load_permissions("../../Data/structured_data\VOC Cochin Slave Transport Permissions 1770-1795 - IISH - Database - 2018 - v1.csv")

E:\UvA\Scriptie\Thesis_code\Annotation\Annotation_helper.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  col = col.str.replace('[^a-zA-Z ]', '')


In [5]:
distances = pd.read_csv(dist_folder + "/distances_0.csv", dtype={'permission_indx': 'uint16', 'transaction_indx': 'uint16', 'SlaafGender': 'bool', 'SlaafNaamNieuw': 'float16', 'BezitterGender': 'bool', 'BezitterVoornaam': 'float16', 'BezitterBeroep': 'float16', 'BezitterAchternaam': 'float16'})
for i in range(1, 99):
    df2 = pd.read_csv(f"{dist_folder}/distances_{i}.csv", dtype={'permission_indx': 'uint16', 'transaction_indx': 'uint16', 'SlaafGender': 'bool', 'SlaafNaamNieuw': 'float16', 'BezitterGender': 'bool', 'BezitterVoornaam': 'float16', 'BezitterBeroep': 'float16', 'BezitterAchternaam': 'float16'})
    distances = pd.concat([distances, df2])
distances.reset_index(inplace=True)

In [6]:
def count_missing_matches(distances, matches):
    count = 0
    real_matches = matches[matches["transaction_indx"] != 'None']
    for i, match in real_matches.iterrows():
        res = distances.query(f"transaction_indx == {match['transaction_indx']} & permission_indx == {match['permission_indx']}").index
        if len(res) == 0:
            count += 1
    return count

In [ ]:
def test_thresholds(distances, matches, column):
    n_matches = matches[matches["transaction_indx"] != 'None'].shape[0]
    data_size = [1]
    n_missing_matches = [0]
    x = [0]
    for i in range(1,11):
        filtered = distances[distances[column] >= i/10]
        n_missing_matches.append(count_missing_matches(filtered, matches)/n_matches)
        x.append(i/10)

        data_size.append(filtered.shape[0]/distances.shape[0])
    return (data_size, n_missing_matches, x)

In [ ]:
blocking_col = 'SlaafNaamNieuw'
blocking_col = 'SlaafNaamNieuw'
data_size, n_missing_matches, x = test_thresholds(distances, matches, blocking_col)
plt.plot(x, data_size, label='relative size of dataset')
plt.plot(x, n_missing_matches, label='relative amount of missing matches')
plt.xlabel("Threshold")
plt.legend()
plt.tight_layout()
plt.savefig("../Figures/Method/Blocking.jpg", bbox_inches="tight")
plt.show()

In [27]:
d = distances
d = d[d["SlaafNaamNieuw"] > 0.6]
d = d[d["SlaafGender"] > 0.1]
# d = d[d["BezitterAchternaam"] > 0.1]
# d = d[d["BezitterVoornaam"] > 0.1]
d = d[d["BezitterGender"] > 0.1]
count_missing_matches(d, matches)
d.shape

(303045, 12)

In [29]:
d = distances
d = d[d["SlaafNaamNieuw"] > 0.6]
count_missing_matches(d, matches)


1

In [15]:
count_missing_matches(distances, matches)

0

In [19]:
distances.shape

(27314400, 12)

In [18]:
d = distances
d = d[d["SlaafNaamNieuw"] > 0.6]
count_missing_matches(d, matches)
d.shape

(366406, 12)

In [10]:
d[['permission_indx', 'transaction_indx']].to_csv("../../Data/structured_data/blocked_possible_matches.csv")